# __Проект: Обучение с учителем: качество модели.__

**Задача**:  построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца, и, используя данные модели и данные о прибыльности клиентов, выделить сегменты покупателей и разработать для них персонализированные предложения. 


**Данные** состоят из четырёх таблиц, в которых содержится информация о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении, о выручке, которую получает магазин с покупателя,о времени, которое покупатель провёл на сайте в течение периода и о среднемесячной прибыли покупателя за последние 3 месяца.

#### Импортируем необходимые библиотеки.

In [ ]:
!pip install -q --upgrade scikit-learn
!pip install -q --upgrade matplotlib
!pip install -q --upgrade seaborn
!pip install -q --upgrade numpy
!pip install -q --upgrade numba

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 2.0.2 which is incompatible.


In [ ]:
!pip install phik
!pip install shap

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import shap
import phik
from phik import phik_matrix

from sklearn.linear_model import (LinearRegression,
                                  LogisticRegression)
from sklearn.model_selection import (train_test_split,
                                    GridSearchCV,
                                    RandomizedSearchCV)
from sklearn.preprocessing import (OneHotEncoder,
                                   StandardScaler,
                                   MinMaxScaler,
                                  LabelEncoder,
                                  OrdinalEncoder)
from sklearn.metrics import (r2_score,
                             roc_auc_score,
                             mean_absolute_error,
                             mean_squared_error,
                             accuracy_score,
                            confusion_matrix,
                            recall_score,
                            precision_score)

from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


## Шаг. Загрузка данных

In [ ]:
try:
    market_file = pd.read_csv('/datasets/market_file.csv')
    market_money = pd.read_csv('/datasets/market_money.csv')
    market_time = pd.read_csv('/datasets/market_time.csv')
    money = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')
except:
    try:
        market_file = pd.read_csv('C:/Users/1/Documents/DS 2024///datasets/market_file.csv')
        market_money = pd.read_csv('C:/Users/1/Documents/DS 2024//datasets/market_money.csv')
        market_time = pd.read_csv('C:/Users/1/Documents/DS 2024//datasets/market_time.csv')
        money = pd.read_csv('C:/Users/1/Documents/DS 2024//datasets/money.csv', sep=';', decimal=',')
    except:
        market_file = pd.read_csv('https://code.s3.yandex.net/datasets///datasets/market_file.csv')
        market_money = pd.read_csv('https://code.s3.yandex.net/datasets//datasets/market_money.csv')
        market_time = pd.read_csv('https://code.s3.yandex.net/datasets//datasets/market_time.csv')
        money = pd.read_csv('https://code.s3.yandex.net/datasets//datasets/money.csv', sep=';', decimal=',')

In [ ]:
market_file.head()

In [ ]:
market_money.head()

In [ ]:
market_time.head()

In [ ]:
money.head()

**Вывод:** Данные в таблицах соответствуют описанию, перейдём к изучению данных и их предобработке.

## Шаг. Предобработка данных

Изучим данные в каждой из четырёх таблиц.

In [ ]:
market_file.info()

В целом таблица выглядит нормально. В столбцы 'Покупательская активность', 'Тип сервиса' и  'Разрешить сообщать'  добавим ' _' между словами в названии для единообразия.

In [ ]:
market_file = market_file.rename(columns = {'Покупательская активность':'Покупательская_активность', 'Тип сервиса':'Тип_сервиса', 'Разрешить сообщать':'Разрешить_сообщать'})

In [ ]:
market_file.head(2)

Рассмотрим стоблец 'Тип_сервиса'.

In [ ]:
market_file['Тип_сервиса'].unique()

Исправим опечатку в названии категорий типа сервиса и избавимся от неявного дубликата.

In [ ]:
market_file['Тип_сервиса'] = market_file['Тип_сервиса'].replace('стандартт', 'стандарт')

In [ ]:
market_file['Тип_сервиса'].unique()

Изучим таблицу на наличие дубликатов и пропущенных занчений.

In [ ]:
market_file.duplicated().sum()

In [ ]:
market_file.isna().sum()

Пропущенных значений и полных дубликатов в первой таблице нет, переходим к изучению данных второй таблицы.

In [ ]:
market_money.info()

Рассмотрим стоблец 'Период'

In [ ]:
market_money['Период'].unique()

Кажется, в названии категорий временного периода закралась ошибка. Исправим ее.

In [ ]:
market_money.isna().sum()

In [ ]:
market_money.duplicated().sum()

Перейдём к изучению данных третьей таблицы.

In [ ]:
market_time.info()

In [ ]:
market_time['Период'].unique()

In [ ]:
market_time['Период'] = market_time['Период'].replace('предыдцщий_месяц', 'предыдущий_месяц')

In [ ]:
market_time['Период'].unique()

In [ ]:
market_time.isna().sum()

In [ ]:
market_time.duplicated().sum()

Пропущенных значений и дубликатов нет, поэтому рассмотрим последнюю таблицу с данными о среднемесячной прибыли покупателя за последние 3 месяца.

In [ ]:
money.info()

In [ ]:
money.isna().sum()

In [ ]:
money.duplicated().sum()

**Вывод**
Мы изучили данные четырёх таблиц. В таблице 'market_file' и 'market_time' были внесены изменения в названия категорий столбцов и исправлены ошибки в названиях категорий. В таблицах полные дубликаты обнаружены не были, и во всех таблицах отсутствуют пропущенные значения - можно приступить к исследовательскому анализу данных.

## Шаг. Исследовательский анализ данных.

Проведём исследовательский анализ данных по каждой таблице.

In [ ]:
market_file.info()

Рассмотрим сначала категориальные призанки, потом - количественные. Начнём с 'покупательной_активности' - признака, который является целевым в нашем исследовании.

In [ ]:
market_file['Покупательская_активность'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Гистограмма покупательной активности')
plt.xlabel('Покупательная активность');

По диаграмме можно сделать вывод, что активность большинства покупателей осталась на прежнем уровне: у 500 человек из 1300 покупательная активность снизилась, у остальных - осталась на прежнем уровне.

In [ ]:
market_file['Тип_сервиса'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Гистограмма распределения типов сервиса среди покупателей')
plt.xlabel('Типа сервиса');

Анализируя распространение типов сервиса, можно заключить, что большей популярностью пользуется 'стандарт', его выбирает 71% покупателей.

In [ ]:
market_file['Разрешить_сообщать'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Гистограмма распределения согласия покупаталей на получение дополнительной рассылки')
plt.xlabel('Разрешать сообщать');

Большинство покупателей не против получать дополнительные сообщения с информацией о товаре.

In [ ]:
market_file['Популярная_категория'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Круговая диаграмма популярности различных категорий товаров');

Одной из самых популярных категорий товаров являются 'товары для детей' (25.4% всех покупок). Немного по популярности уступают 'домашний текстиль'(19.3%) и 'косметика и аксесуары' (17.2%). Наименее популярной категорией стала 'кухонная посуда' (10.6%)

Перейдём к анализу количественных признаков первой таблицы.

In [ ]:
market_file.info()

In [ ]:
def stat(column):
    return column.median(), column.describe(), column.hist(bins=30), plt.title('Столбчатая гистограмма признака'), plt.xlabel('Признак'), plt.ylabel('Количество');

In [ ]:
stat(market_file['Маркет_актив_6_мес'])

'Маркет_актив_6_мес' - это число рассылок, звонков, показов рекламы и прочего, которое приходилось на клиента за последние 6 месяцев. Среднее значение этого признака близко к значению медианы, что может быть косвенным свидетельством отсутствия экстремалных значений. Проверим это с помощью ящичковой диаграммы.

In [ ]:
market_file.boxplot('Маркет_актив_6_мес')
plt.title('Ящичковая диаграмма Маркет_актив за полгода');

Оба графика показывают наличие низких значений рассылок, но я предлагаю эти данные не удалять, они не являются экстремальными.

In [ ]:
stat(market_file['Маркет_актив_тек_мес'])

Для признака 'Маркет_актив_тек_мес' среднее значение и медиана практически совпадают. Также по графику можно выделить три группы покупателей, получивших дополнительную информацию о товарах в текущем месяце.

In [ ]:
stat(market_file['Длительность'])

'Длительность' показывает, сколько дней прошло с момента регистрации покупателя на сайте. Значение медианы немного выше среднего значения, также довольно высокое стандартное отклонение. Построим столбчатую диаграмму.

In [ ]:
market_file.boxplot('Длительность')
plt.title('Ящичковая диаграмма длительности регистрации на сайте');

In [ ]:
stat(market_file['Акционные_покупки'])

По значениям описательной статистики признака видно сильное отличие среднего и медианного значений. По графику гистограммы можно выделить две группы покупетелей: те, у кого доля акционных покупок составляет больше 0.5 и тех, у кого эта доля ментше 0.5

In [ ]:
market_file.boxplot('Акционные_покупки')
plt.title('Ящичковая диаграмма среднемесячного количества доли акционных покупок');

In [ ]:
market_file.loc[market_file['Акционные_покупки'] > 0.6]['Акционные_покупки'].count()

По графику ящичковой диаграммы мы видим целый ряд значений, которые превышают "усики". Но я предлагаю их не удалять, т.к. они не являются нереалистичными.

In [ ]:
stat(market_file['Средний_просмотр_категорий_за_визит'])

По гистограмме признака можно сделать вывод, что в среднем за последний месяц покупатель за визит на сайт больше всего рассматривал до 2 до 4 категорий товаров.

In [ ]:
(market_file.loc[(market_file['Средний_просмотр_категорий_за_визит'] >= 2) & (market_file['Средний_просмотр_категорий_за_визит'] < 5) ]['Средний_просмотр_категорий_за_визит'].count()/ market_file['Средний_просмотр_категорий_за_визит'].count())*100

Доля таких покупателей составляет 71.6%

In [ ]:
stat(market_file['Неоплаченные_продукты_штук_квартал'])

In [ ]:
market_file.boxplot('Неоплаченные_продукты_штук_квартал')
plt.title('Ящичковая диаграмма неоплаченных товаров за последний квартал');

In [ ]:
stat(market_file['Ошибка_сервиса'])

In [ ]:
market_file.boxplot('Ошибка_сервиса')
plt.title('Ящичковая диаграмма числа сбоев на сайте');

In [ ]:
stat(market_file['Страниц_за_визит'])

In [ ]:
market_file.boxplot('Страниц_за_визит')
plt.title('Ящичковая диаграмма соеднего количества страниц за визит');

In [ ]:
numeric_features = ['Маркет_актив_6_мес', 'Маркет_актив_тек_мес', 'Длительность', 'Акционные_покупки', 'Средний_просмотр_категорий_за_визит', 'Ошибка_сервиса', 'Страниц_за_визит', 'Неоплаченные_продукты_штук_квартал']
category_features = ['Тип_сервиса', 'Разрешить_сообщать', 'Популярная_категория']
for category in category_features:
    sns.pairplot(market_file, x_vars=numeric_features, y_vars='Покупательская_активность', hue=category)

Перейдём к исследованию данных второй таблицы.

In [ ]:
stat(market_money['Выручка'])

Среднее значение сильно превышает медианное, а также велико стандартное отклонение. Построим ящичковую диаграмму для этого признака.

In [ ]:
market_money.boxplot('Выручка')
plt.title('Ящичковая диаграмма суммы выручки');

Одно знаение очень сильно выбивается, рассмотрим его подробнее.

In [ ]:
market_money.loc[market_money['Выручка'] > 50000]

Это значение является экстремальным значением, я предлагаю удалить эту цифру, чтобы она не искажала модель в дальнейшем, и заменить ее средним значением для этого покупателя.

In [ ]:
market_money.loc[market_money['id'] == 215380]

In [ ]:
market_money.loc[(market_money['id'] == 215380) & (market_money['Период'] != 'текущий_месяц')].mean()

In [ ]:
market_money['Выручка'] = market_money['Выручка'].replace(106862.2, 5564.0)

In [ ]:
market_money.loc[market_money['id'] == 215380]

Повторим исследование описательной статистики для этого признака после внесения изменений.

In [ ]:
stat(market_money['Выручка'])

Сгруппируем данные по выручке по временному периоду и построим график.

In [ ]:
market_money.groupby('Период')['Выручка'].hist(bins=20, figsize=(7,5), legend='Период', alpha=0.5)
plt.xlabel('Сумма выручки');

In [ ]:
market_money.boxplot('Выручка')
plt.title('Ящичковая диаграмма суммы выручки');

In [ ]:
market_money['Период'].value_counts().plot(kind='pie', figsize=(5,5), autopct='%.1f')
plt.title('Гистограмма периодов, за которые была зафиксирована выручка')
plt.xlabel('Периоды');

За все месяцы было зафиксирвоано одинаковое количество пользователей, покупавших товары.

In [ ]:
stat(market_time['минут'])

In [ ]:
market_time.groupby('Период')['минут'].hist(bins=20, figsize=(7,5), legend='Период', alpha=0.5)
plt.xlabel('Количество минут, проведённое на сайте');

In [ ]:
market_time.boxplot('минут')
plt.title('Ящичковая диаграмма времени, проведённого на сайте, в минутах');

По статистическим показателям выбросов и экстремальных значений в признаке "минут" нет.

In [ ]:
market_time['Период'].value_counts().plot(kind='pie', figsize=(5,5), autopct='%.1f')
plt.title('Гистограмма периодов, за которые учтено общее время')
plt.xlabel('Период');

Два наблюдаемых периода соответствуют друг другу по количеству покупателей.

In [ ]:
stat(money['Прибыль'])

Столбчатый график по признаку "Прибыль" очень напоминает график нормального распределения.

In [ ]:
money.boxplot('Прибыль')
plt.title('Ящичковая диаграмма прибыли');

Нам необходимо отобрать клиентов с покупательной активностью не менее трёх месяцев, то есть таких, которые что-либо покупали в этот период.

In [ ]:
tempory_file = market_money.query('Выручка == 0')

In [ ]:
tempory_file.head(10)

Мы обнаружили трёх клиентов, которые не были активны на протяжение трёх последних месяцев. Удалим их из всех таблиц для того, чтобы они не влияли на результаты дальнейших этапов анализа.

In [ ]:
zeroes = [215348, 215357, 215359]

In [ ]:
market_money_new = market_money[market_money.id.isin (zeroes) == False ]

In [ ]:
market_money_new.info()

In [ ]:
market_file_new = market_file[market_file.id.isin (zeroes) == False ]

In [ ]:
market_time_new = market_time[market_time.id.isin (zeroes) == False ]

In [ ]:
money_new = money[money.id.isin (zeroes) == False ]

**Вывод**

Мы провели исследовательский анализ данных. Нами было выявлено, что активность большинства покупателей осталась на прежнем уровне: у 500 человек из 1300 покупательная активность снизилась, у остальных - осталась на прежнем уровне.Анализируя распространение типов сервиса, можно заключить, что большей популярностью пользуется 'стандарт', его выбирает 71% покупателей.
Большинство покупателей не против получать дополнительные сообщения с информацией о товаре.
Одной из самых популярных категорий товаров являются 'товары для детей' (25.4% всех покупок). Немного по популярности уступают 'домашний текстиль'(19.3%) и 'косметика и аксесуары' (17.2%). Наименее популярной категорией стала 'кухонная посуда' (10.6%)
'Маркет_актив6мес' - это число рассылок, звонков, показов рекламы и прочего, которое приходилось на клиента за последние 6 месяцев. Среднее значение этого признака близко к значению медианы, что может быть косвенным свидетельством отсутствия экстремалных значений. 
Оба графика показывают наличие низких значений рассылок, но я предлагаю жти данные не удалять, они не являются экстремальными.
Для признака 'Маркет_актив_тек_мес' среднее значение и медиана практически совпадают. Также по графику можно выделить три группы покупателей, получивших дополнительную информацию о товарах в текущем месяце.
По графику гистограммы можно выделить две группы покупетелей: те, у кого доля акционных покупок составляет больше 0.5 и тех, у кого эта доля ментше 0.5
По графику ящичковой диаграммы среднемесячного количества доли акционных покупок мы видим целый ряд значений, которые превышают "усики". Но я предлагаю их не удалять, т.к. они не являются нереалистичными.По гистограмме признака можно сделать вывод, что в среднем за последний месяц покупатель за визит на сайт больше всего рассматривал до 2 до 4 категорий твоаров. Доля таких покупателей составляет 71.6%Удалили одно экстремальное значение по "выручке".

Также мы отобрали клиентов с покупательной активностью не менее трёх месяцев, исключив из анализа тех, кто хотя бы раз не делал покупки за этот трёхмесячный период.

## Шаг. Объединение таблиц

Объединим все таблицы последовательно, кроме money. Но сначала вынесем временные периоды в таблицах market_money и market_time в столбцы, чтобы сделать все таблицы равными между собой по количеству строк (1300 против 3900 и 2600 в исходных таблицах).

In [ ]:
pivot_market_money = market_money_new.pivot(index='id', columns='Период', values='Выручка').reset_index()

In [ ]:
pivot_market_money.head()

Преобразуем таким же образом таблицу market_time_new.

In [ ]:
pivot_market_time_ = market_time_new.pivot(index='id', columns='Период', values='минут')

In [ ]:
data_1 = market_file_new.merge(pivot_market_money, on='id', how='left')

In [ ]:
data_2 = data_1.merge(pivot_market_time_, on=['id'], how='left')

Проверим, адекватно ли выглядит итоговая таблица после слияния.

In [ ]:
data_2.info()

In [ ]:
data_2.head(3)

In [ ]:
data_2 = data_2.rename(columns = {'предыдущий_месяц_x':'Выручка_02_месяц', 'препредыдущий_месяц':'Выручка_03_месяц',
                              'текущий_месяц_x':'Выручка_01_месяц', 'предыдущий_месяц_y':'Минут_02_месяц',
                             'текущий_месяц_y':'Минут_01_месяц'})

In [ ]:
data_2.head(3)

In [ ]:
data = data_2.set_index('id')

**Вывод**
Мы объединили все таблицы, которые понадобятся нам для дальнейшего моделирвоания, а также создали отдельный столбец по времени, проведённом на сайте, и выручке для каждого временного периода. Приступаем к корреляционному анализу!

##  Шаг. Корреляционный анализ

Изучим взаимосвязь между признаками, рассчитав между количественными признаками коэффициенты корреляции, воспользуемся библиотекой phix, методом phix_matrix.

In [ ]:
data_1 = data.drop(['Покупательская_активность', 'Тип_сервиса', 'Разрешить_сообщать', 'Популярная_категория'], axis=1)
data_1.phik_matrix()

Попробуем изобразить эту таблицу с помощью тепловой карты для лучшего восприятия.

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(data_1.phik_matrix(), annot=True, cmap='coolwarm')
plt.title('Коэффициенты корреляции признаков')
plt.show();

Для трактовки силы связи между признаками будем использовать шкалу Чеддока. Таким образом, очень высокая линейная связь отмечается между выручкой в текущем месяце и выручкой в предыдущем месяце (коэффициент корреляции равен 0.84),что может свидетельствовать о мультиколлинеарности этих признаков. Пока мы принимаем решение не убирать один из этих признаков из таблицы, чтобы избавиться от мультиколлинеарности.
Умеренная связь выражена между признаками 'страниц_за_визит' и время, проведённое на саёте в предыдущем месяце (коэффициент коллеляции = 0.48), выручкой за препредыдущий месяц и временем, проведённом на сайте в предыдущий месяц (0.42), а также между признаками 'страниц_за_визит' и 'неоплаченные_продукты_штук_квартал' (0.36) и 'Маркет_актив_6_мес' (0.39).

**Вывод** 
Мы провели корреляционный анализ, в ходе которого выявили очень высокую линейную связь и предполагаемую мультиколлинеарность между выручкой за текущий и предыдущий месяц (коэффициент корреляции равен 0.84). Умеренная связь выражена между признаками 'страниц_за_визит' и время, проведённое на сайте в предыдущем месяце (коэффициент коллеляции = 0.48), выручкой за препредыдущий месяц и временем, проведённом на сайте в предыдущий месяц (0.42), а также между признаками 'страниц_за_визит' и 'неоплаченные_продукты_штук_квартал' (0.36) и 'Маркет_актив6мес' (0.39).

In [ ]:
g = sns.PairGrid(data, x_vars='Выручка_01_месяц', y_vars='Выручка_02_месяц', hue='Покупательская_активность')
g.map(sns.scatterplot);

In [ ]:
m = sns.PairGrid(data_2, x_vars='id', y_vars='Акционные_покупки', hue='Покупательская_активность')
m.map(sns.scatterplot);

## Шаг. Использование пайплайнов

Подготовим данные для обучения модели, используя  ColumnTransformer. Количественные и категориальные признаки обработаем раздельно, для кодирования категориальных признаков используем OneHotEncoder и OrdinalEncoder, а для масштабирвоания количественных - StandardScaler и MinMaxScaler.
Создадим списки с названиями признаков и пайплайны для каждого кодирования и шага, а также общий пайплайн для подготовки данных. 
Но сначала для удобства закодируем знчаения признака 'Покупательская_активность' в числовой формат.

In [ ]:
data['Покупательская_активность'].head(5)

In [ ]:
le = LabelEncoder()
le.fit(data['Покупательская_активность'])
le.classes_[1], le.classes_[0] = le.classes_[0], le.classes_[1]
data['Покупательская_активность'] = le.transform(data['Покупательская_активность'])

In [ ]:
data['Покупательская_активность'].head()

In [ ]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

X = data.drop(['Покупательская_активность'], axis=1)
y = data['Покупательская_активность']

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = y)


X_train.shape, X_test.shape

ohe_columns = [
    'Популярная_категория', 'Разрешить_сообщать'
]
ord_columns = [
    'Тип_сервиса'
]
num_columns = ['Маркет_актив_6_мес', 'Маркет_актив_тек_мес', 'Длительность', 'Акционные_покупки',
               'Средний_просмотр_категорий_за_визит','Неоплаченные_продукты_штук_квартал',
               'Ошибка_сервиса', 'Страниц_за_визит', 'Выручка_01_месяц', 'Выручка_02_месяц',
               'Выручка_03_месяц', 'Минут_01_месяц', 'Минут_02_месяц']

ohe_pipe = Pipeline(
    [
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='error',  sparse_output=False)
        )
    ]
)

ord_pipe = Pipeline(
    [
        (
            'ord',
            OrdinalEncoder(
                categories=[
                    ['премиум', 'стандарт'],
                ], 
                handle_unknown='use_encoded_value',
                unknown_value=np.nan
            )
        )
    ]
)

data_preprocessor = ColumnTransformer([
    ('ohe', ohe_pipe, ohe_columns),
    ('ord', ord_pipe, ord_columns),
    ('num', MinMaxScaler(), num_columns)],
    remainder='passthrough'
)

Создадим итоговый пайплайн

In [ ]:
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

 Обучим четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и  SVC(). Для каждой из них попробуем подобрать как минимум один гиперпараметр. В качестве метрики будем использовать roc-auc, потому что она отображает истинную и постоянную способность модели к прогнозированию, устойчива к несбалансированным классам и может быть использована для сравнения различных моделей классификации.

In [ ]:
param_distributions = [
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(1, 25),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
    },
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 15),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]  
    },
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', 
            penalty='l1'
        )],
        'models__C': range(1,5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()]  
    },
    {   'models': [SVC(random_state=RANDOM_STATE, probability=True)], 
        'models__kernel': ['poly', 'rbf', 'sigmoid'],
        'models__degree': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler()] 
}
]

randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_distributions=param_distributions, 
    scoring='roc_auc', 
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

Командой grid_search.cv_results_ получим все результаты и сделаем из них датафрейм. Выведем четыре столбца, отсортируем по rank_test_score — рейтингу качества моделей:

In [ ]:
result = pd.DataFrame(randomized_search.cv_results_)
print(result[
    ['rank_test_score', 'param_models', 'mean_test_score','params']
].sort_values('rank_test_score')) 

Получается, модель с наилучшими параметрами - KNeighborsClassifier(n_neighbors=14). Сохраним её.

In [ ]:
y_proba = randomized_search.best_estimator_.predict_proba(X_test)[:, 0]

In [ ]:
table = pd.DataFrame(zip(y_test, y_proba),
             columns = ['y_valid', 'y_proba']).sort_values(by='y_proba',ascending=False)

table.head()

**Вывод**
Мы подготовили данные для обучения модели, используя ColumnTransformer. Для удобства работы мы закодировали знчаения признака 'Покупательская_активность' в числовой формат. Количественные и категориальные признаки обработали раздельно, для кодирования категориальных признаков использовали OneHotEncoder и OrdinalEncoder, а для масштабирвоания количественных - StandardScaler и MinMaxScaler. Создали списки с названиями признаков и пайплайны для каждого кодирования и шага, а также общий пайплайн для подготовки данных. Мы обучили четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и SVC(). Для каждой из них подобрали как минимум один гиперпараметр, в качестве метрики использовали roc-auc, которая отображаеть истинную и постоянную способность модели к прогнозированию.
Лучшей оказалась модель с наилучшими параметрами - KNeighborsClassifier(n_neighbors=14).

## Шаг. Анализ важности признаков

Оценим важность признаков для лучшей модели, которую мы получили в предыдущем шаге. Для этого сначала подготовим данные и используем графики метода SHAP. 
Для того, чтобы определить, какие признаки являются наиболее важными для прогнозов, выдаваемых моделью, используем столбчатую диаграмму (shap.plots.bar).

X_train_2 = pd.DataFrame(data_preprocessor.fit_transform(X_train), columns=data_preprocessor.get_feature_names_out() )
X_test_2 = pd.DataFrame(data_preprocessor.transform(X_test), columns=data_preprocessor.get_feature_names_out())

model = randomized_search.best_estimator_.named_steps['models']
model.fit(X_train_2.values, y_train)

explainer = shap.KernelExplainer(lambda x: model.predict_proba(x), shap.sample(X_train_2, 10))
shap_values = explainer(shap.sample(X_test_2, 10))
shap.plots.bar(shap_values[:, :, 0], max_display=21)

График помогает понять, какие признаки являются наиболее важными для работы нашей модели. Получается, самое большое среднее значение shap имеет признак "акционные покупки". Это значит, что этот признак оказывает наиболее сильное воздействие на прогнозы модели. На втором и третьем месте по "важности" - количество страниц за визит и количество времени, проведённое на сайте в текущем месяце. Наименее значимыми для модели получились признаки: выручка за текущий и предыдущий месяцы, ошибки сервиса, популярные категории (косметика и аксесуары) и неоплаченные продукты за квартал. 
При моделировании и принятии бизнес-решений можно использовать информацию, полученную при анализе важности признаков. Так, для более точного результата прогноза можно исключить из исследования наименее важные признаки, оставив наиболее ценные.

explainer = shap.KernelExplainer(lambda x: model.predict_proba(x), shap.sample(X_train_2))
shap_values = explainer.shap_values(shap.sample(X_test_2))
shap_obj = explainer(shap.sample(X_test_2))
shap.plots.beeswarm(shap_obj[:,:,1], max_display=21)

Исследуя SHAP‑значения на этой диаграмме, мы можем исследовать природу взаимоотношений между признаками и спрогнозированной покупательской активностью. Такие признаки, как количество страниц за визит, время, проведённое на сайте за текущий и прошлый месяц, средний просмотр категорий за визит растут по мере увеличения покупательской активности. Мы можем заметить и обратную ситуацию: чем больше значения признаков - тем меньше SHAP‑значения. Это наблюдение указывает на то, что более высокие значения 'неоплаченные продукты_шт_квартал', 'длительность' и 'выручка за текущий месяц' связаны с более низкой спрогнозированной покупательной активностью.

**Вывод**
Для анализа важности призанков мы использовали графики метода shap. Столбчатый график помогает понять, какие признаки являются наиболее важными для работы модели. Самое большое среднее значение shap имеет признак "акционные покупки", который оказывает наиболее сильное воздействие на прогнозы модели. На втором и третьем месте по "важности" - количество страниц за визит и количество времени, проведённое на сайте в текущем месяце. Наименее значимыми для модели получились признаки: выручка за текущий и предыдущий месяцы, ошибки сервиса, популярные категории (косметика и аксесуары) и неоплаченные продукты за квартал. При моделировании и принятии бизнес-решений можно использовать информацию, полученную при анализе важности признаков. Так, для более точного результата прогноза можно исключить из исследования наименее важные признаки, оставив наиболее ценные.

## Шаг. Сегментация покупателей

После получения прогнозов по вероятности снижения покупательской активности, рассмотрим покупаталей с точки зрения получаемой от них прибыли. Для этого построим диаграмму разброса.

In [ ]:
probability = pd.DataFrame({'id': X_test.index, 'Вероятность_снижения': y_proba})

In [ ]:
table_2 = probability.merge(money_new, on=['id'], how='left')

In [ ]:
table_2.head(3)

In [ ]:
table_2.plot(kind='scatter', x='Вероятность_снижения', y='Прибыль',  figsize=(12,5), c='blue')
plt.title('Диаграмма разброса прибыли в зависимости от вероятности снижения покупательской активности')
plt.ticklabel_format(style='plain')
plt.xlabel('Вероятность снижения покупательской активности')
plt.ylabel('Прибыль');

Рассмотрим группу клиентов с наиболее высокой прибыльностью и высокой вероятностью снижения покупательской активности.

In [ ]:
_f1 = table_2['Вероятность_снижения'] < 0.25
_f2 = table_2['Прибыль'] >= 3
low_activity = table_2[_f1 & _f2].sort_values(by='Вероятность_снижения', ascending=False)
low_activity.shape[0]

In [ ]:
low_activity.head()

Таких клиентов оказалось 156 человек.Рассмотрим, на какие категории товаров они тратят больше всего и меньше всего, как меняются их траты в течение трёх месяцев, проверим, по каким категориям товров траты проседают, а по каким наоборот увеличиваются. Добавим информацию об этом из нашей основной таблицы.

In [ ]:
low_activity_view = low_activity.merge(data[['Популярная_категория','Выручка_01_месяц', 'Выручка_02_месяц', 'Выручка_03_месяц']], on=['id'], how='left')

In [ ]:
low_activity_view.head()

In [ ]:
low_activity_view['Популярная_категория'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Круговая диаграмма популярности различных категорий товаров среди клиентов с высокой вероятностью снижения покупательской активности');

Наибольшей популярностью среди покупателей жтой группы пользовались такие категории товаров, как домашний текстиль, товары для детей и мелкая бытовая техника и электроника. Наименьшей популярностью пользовалась  косметика и аксесуары и кухонная посуда. Интересно посмотреь динамику покупок товаров этих категорий по времени.

Рассмотрим поведение группу покупаталей с высокой прибыльностью, но для которых наша модель предсказала другой результат: тех, чья покупательская активность осталась на прежнем уровне.

In [ ]:
_f1 = table_2['Вероятность_снижения'] > 0.75
_f2 = table_2['Прибыль'] >= 3
high_activity = table_2[_f1 & _f2].sort_values(by='Вероятность_снижения', ascending=False)
high_activity.shape[0]

In [ ]:
high_activity_view = high_activity.merge(data[['Популярная_категория','Выручка_01_месяц', 'Выручка_02_месяц', 'Выручка_03_месяц']], on=['id'], how='left')

In [ ]:
high_activity_view['Популярная_категория'].value_counts().plot(kind='pie', figsize=(7,7), autopct='%.1f')
plt.title('Круговая диаграмма популярности различных категорий товаров среди клиентов с низкой вероятностью снижения покупательской активности');

В этой группе покупателей почти в два раза меньше (72 против 156), и при этом популярными являются совсем другие категории товаров. Построим графики для выручси за текущий и предыдущий месяц в обеих рассматриваемых группах.

In [ ]:
agg_func = {
    'Выручка_01_месяц': ['median'],
    'Выручка_02_месяц': ['median'],
    'Выручка_03_месяц': ['median']
}
low_activity_view.groupby('Популярная_категория').agg(agg_func).head(6)

In [ ]:
high_activity_view.groupby('Популярная_категория').agg(agg_func).head(6)

Выделенная нами группа покупателей с высокой вероятностью снижения покупательной активности имеет лучшие показатели по прибыльности, чем группа спокупательской активностью на прежнем уровне. Для повышения покупательской активности рассматриваемой группы, мы рекомендуем ввести акции для популярных категорий товаров, таких, как домашний текстиль, товары для детей и мелкая бытовая техника и электроника.

**Вывод**
По результатам моделирования и после оценки важности признаков с помощью графиков shap, мы выяснили, что наиболее сильное воздействие на прогнозы модели оказывал признак "акционные покупки". Используя результаты моделирования и данные о прибыльности покупателей, мы для дальнейшего анализа выбрали группу клиентов с высокой прибыльностью и высокой вероятностью снижения покупательской активности. Мы исследовали, на какие категории товаров они тратят больше всего и меньше всего, изучили, как менялись траты в течение трёх месяцев и по каким категориям траты проседали, а по каким наоборот увеличивались. Мы сравнили изучаемую группу с контр-группой и в качестве рекомендаций для повышения покупательской активности мы предлагаем ввести акции на категории товаров, на которые они любят тратить больше всего (товары для детей, домашний текстиль), чтобы увеличить их покупательскую активность. Изучив разницу между покупками в разные месяцы, мы можем предложить ввести акции на категорию "техника дял красоты и здоровья", поскольку продажи в этой группе товаров почти не меняются. Возможно, это потому, что техника чаще покумается на долгий срок, а, возможно, потому, что на данную категорию требуются акции для увеличения покупательской активности.

## Шаг. Общий вывод.

Сделайте общий вывод:
опишите задачу;
опишите исходные данные и проведённую предобработку;
напишите, что вы сделали для поиска лучшей модели;
укажите лучшую модель;
добавьте выводы и дополнительные предложения для выбранного сегмента покупателей.

В нашу задачу входило изучить данные и по ним построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца, и, используя данные модели и данные о прибыльности клиентов, выделить сегменты покупателей и разработать для них персонализированные предложения.

Данные представляли собой четыре таблицы, в которых содержалась информация о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении, о выручке, которую получает магазин с покупателя,о времени, которое покупатель провёл на сайте в течение периода и о среднемесячной прибыли покупателя за последние 3 месяца. Мы изучили данные таблиц. В таблице 'market_file' и 'market_time' были внесены изменения в названия категорий столбцов и исправлены ошибки в названиях категорий. В таблицах полные дубликаты обнаружены не были, и во всех таблицах отсутствовали пропущенные значения

Мы провели исследовательский анализ данных. Было выявлено, что активность большинства покупателей осталась на прежнем уровне: у 500 человек из 1300 покупательная активность снизилась, у остальных - осталась на прежнем уровне.Анализируя распространение типов сервиса, мы заключили, что большей популярностью пользуется 'стандарт', его выбирает 71% покупателей. Большинство покупателей не против получать дополнительные сообщения с информацией о товаре. Одной из самых популярных категорий товаров являются 'товары для детей' (25.4% всех покупок). Немного по популярности уступают 'домашний текстиль'(19.3%) и 'косметика и аксесуары' (17.2%). Наименее популярной категорией стала 'кухонная посуда' (10.6%) 'Маркет_актив6мес' - это число рассылок, звонков, показов рекламы и прочего, которое приходилось на клиента за последние 6 месяцев. Среднее значение этого признака близко к значению медианы, что может быть косвенным свидетельством отсутствия экстремалных значений. Оба графика показывают наличие низких значений рассылок, но я предлагаю жти данные не удалять, они не являются экстремальными. Для признака 'Маркет_актив_тек_мес' среднее значение и медиана практически совпадают. Также по графику можно выделить три группы покупателей, получивших дополнительную информацию о товарах в текущем месяце. По графику гистограммы можно выделить две группы покупетелей: те, у кого доля акционных покупок составляет больше 0.5 и тех, у кого эта доля ментше 0.5 По графику ящичковой диаграммы среднемесячного количества доли акционных покупок мы видим целый ряд значений, которые превышают "усики". Но я предлагаю их не удалять, т.к. они не являются нереалистичными.По гистограмме признака можно сделать вывод, что в среднем за последний месяц покупатель за визит на сайт больше всего рассматривал до 2 до 4 категорий твоаров. Доля таких покупателей составляет 71.6%Удалили одно экстремальное значение по "выручке".

Также мы отобрали клиентов с покупательной активностью не менее трёх месяцев, исключив из анализа тех, кто хотя бы раз не делал покупки за этот трёхмесячный период.
В ходе корреляционного анализа мы выявили очень высокую линейную связь между выручкой за текущий и предыдущий месяц (коэффициент корреляции равен 0.84). Умеренная связь выражена между признаками 'страниц_за_визит' и время, проведённое на сайте в предыдущем месяце (коэффициент коллеляции = 0.48), выручкой за препредыдущий месяц и временем, проведённом на сайте в предыдущий месяц (0.42), а также между признаками 'страниц_за_визит' и 'неоплаченные_продукты_штук_квартал' (0.36) и 'Маркет_актив6мес' (0.39).

Далее мы подготовили данные для обучения модели, используя ColumnTransformer. Для удобства работы мы закодировали знчаения признака 'Покупательская_активность' в числовой формат. Количественные и категориальные признаки обработали раздельно, для кодирования категориальных признаков использовали OneHotEncoder и OrdinalEncoder, а для масштабирвоания количественных - StandardScaler и MinMaxScaler. Создали списки с названиями признаков и пайплайны для каждого кодирования и шага, а также общий пайплайн для подготовки данных. Мы обучили четыре модели: KNeighborsClassifier(), DecisionTreeClassifier(), LogisticRegression() и SVC(). Для каждой из них подобрали как минимум один гиперпараметр, в качестве метрики использовали roc-auc, которая отображаеть истинную и постоянную способность модели к прогнозированию. Лучшей оказалась модель с наилучшими параметрами - KNeighborsClassifier(n_neighbors=14).

Для анализа важности призанков мы использовали графики метода shap. Столбчатый график помогает понять, какие признаки являются наиболее важными для работы модели. Самое большое среднее значение shap имеет признак "акционные покупки", который оказывает наиболее сильное воздействие на прогнозы модели. На втором и третьем месте по "важности" - количество страниц за визит и количество времени, проведённое на сайте в текущем месяце. Наименее значимыми для модели получились признаки: выручка за текущий и предыдущий месяцы, ошибки сервиса, популярные категории (косметика и аксесуары) и неоплаченные продукты за квартал. При моделировании и принятии бизнес-решений можно использовать информацию, полученную при анализе важности признаков. Так, для более точного результата прогноза можно исключить из исследования наименее важные признаки, оставив наиболее ценные.

По результатам моделирования и после оценки важности признаков с помощью графиков shap, мы выяснили, что наиболее сильное воздействие на прогнозы модели оказывал признак "акционные покупки". Используя результаты моделирования и данные о прибыльности покупателей, мы для дальнейшего анализа выбрали группу клиентов с высокой прибыльностью и высокой вероятностью снижения покупательской активности. Мы исследовали, на какие категории товаров они тратят больше всего и меньше всего, изучили, как менялись траты в течение трёх месяцев и по каким категориям траты проседали, а по каким наоборот увеличивались. Мы сравнили изучаемую группу с контр-группой и в качестве рекомендаций для повышения покупательской активности мы предлагаем ввести акции на категории товаров, на которые они любят тратить больше всего (товары для детей, домашний текстиль), чтобы увеличить их покупательскую активность. Изучив разницу между покупками в разные месяцы, мы можем предложить ввести акции на категорию "техника дял красоты и здоровья", поскольку продажи в этой группе товаров почти не меняются. Возможно, это потому, что техника чаще покумается на долгий срок, а, возможно, потому, что на данную категорию требуются акции для увеличения покупательской активности.